<a href="https://colab.research.google.com/github/shnajeeb/Hospital-CFM/blob/main/Test_Hospital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Step 1: Install Streamlit and pyngrok
!pip install streamlit pyngrok

In [6]:


# Step 2: Write the Streamlit app code into a file
%%writefile app.py
import streamlit as st
import ezdxf
import cv2
import pytesseract
from PIL import Image
import numpy as np

# Function to process DXF files (.dxf)
def process_dxf(file):
    try:
        doc = ezdxf.readfile(file)
        compliance_issues = []
        for entity in doc.modelspace().query('LINE'):
            length = entity.dxf.length
            if length < 5:
                compliance_issues.append(f"Short duct found at coordinates: {entity.dxf.start} -> {entity.dxf.end}")
        return compliance_issues
    except Exception as e:
        return [f"Error processing DXF file: {str(e)}"]

# Function to process image files (e.g., JPG, PNG) with OpenCV & OCR
def process_image(image):
    try:
        gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
        text = pytesseract.image_to_string(gray)
        compliance_issues = []
        if "duct" in text.lower():
            compliance_issues.append("Found 'duct' in the design text. Check duct placement.")
        if "vent" in text.lower():
            compliance_issues.append("Found 'vent' in the design text. Ensure proper ventilation placement.")
        return compliance_issues
    except Exception as e:
        return [f"Error processing image: {str(e)}"]

# Streamlit UI setup
st.title("HVAC Compliance Checker for OSHPD Hospital Design")
st.write("Upload your HVAC design drawing (DXF, PNG, JPG) to check for compliance with OSHPD standards.")

uploaded_file = st.file_uploader("Upload a drawing", type=["jpg", "png", "pdf", "dxf"])

if uploaded_file is not None:
    file_extension = uploaded_file.name.split('.')[-1].lower()

    if file_extension == 'dxf':
        st.write("Processing CAD (DXF) file...")
        compliance_results = process_dxf(uploaded_file)

    elif file_extension in ['jpg', 'png']:
        st.write("Processing image file...")
        image = Image.open(uploaded_file)
        compliance_results = process_image(image)

    if compliance_results:
        st.write("Compliance Issues Found:")
        for issue in compliance_results:
            st.warning(issue)
    else:
        st.success("No compliance issues detected!")

# Step 3: Create an ngrok tunnel and run Streamlit
from pyngrok import ngrok
import os

# Set port
port = 8501

# Start ngrok tunnel
public_url = ngrok.connect(port)

# Run Streamlit app in the background
os.system('streamlit run app.py &')

# Display the public URL
print(f"Streamlit app is live at: {public_url}")


Overwriting app.py
